In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from ray import tune
import numpy as np
import pandas as pd
import os
import datetime
from ray.tune.suggest.bayesopt import BayesOptSearch
from one_region_simu import *
from evaluation import *
import pdb
import sys

In [3]:
data_folder = os.path.join(os.getcwd(), 'data')
exp_name = '20220608_trial0'
ray_folder = os.path.join(data_folder,'ray_results')
exp_folder = os.path.join(ray_folder, exp_name)
restore_folder = exp_folder
os.makedirs(exp_folder, exist_ok=True)
# os.environ['RAY_PICKLE_VERBOSE_DEBUG']='1'

In [4]:
config={'ntau':tune.uniform(0.0000001,10),
        'nsig': tune.uniform(0.001,0.01),
        "c_ee": tune.uniform(12.5, 13.5),
        "c_ei": tune.uniform(7, 10),
        'c_ie': tune.uniform(6, 18),
        'c_ii': tune.uniform(7, 10),
        'tau_e': tune.uniform(0, 150),
        'tau_i': tune.uniform(0, 150),
        'r_e': tune.uniform(0.5, 2),
        'r_i': tune.uniform(0.5, 2)
        }

def run_model(config):
    params = config_params({"c_ee":config['c_ee'],'c_ei':config['c_ei'],'c_ie':config['c_ie'],'c_ii':config['c_ii'],
                   'tau_e':config['tau_e'],'tau_i':config['tau_i'],'r_e':config['r_e'],'r_i':config['r_i']})
    score = eva_limit_cycle(params)
    if not score:
        region = config_one_region()
        surface = config_surface(region)
        sim = config_simulator(params, region, surface)
        result_name = run_simulation(sim, params, check_point=12500,data_folder=data_folder)
        score = get_score(result_name)
    tune.report(score=score)

In [5]:
# # Customize the logger
from typing import Dict, List
import json
from ray.tune.logger import LoggerCallback
from tensorboardX import SummaryWriter
from shutil import rmtree

class CustomLoggerCallback(LoggerCallback):
    """Put the results of all trails in the same file"""

    def __init__(self, filefolder, filename: str = "log.txt", bayesopt=None):
        self._filename = filename
        self._filefolder = filefolder
        self._filepath = os.path.join(filefolder, self._filename)
        self.bayesopt = bayesopt
        self.waive_params = ["time_this_iter_s","timesteps_total","episodes_total","timestamp", "pid",
                 "hostname", "node_ip", "timesteps_since_restore", "iterations_since_restore", "warmup_time"]
        
    def log_trial_start(self, trial: "Trial"):
        self._file = open(self._filepath, "at")
        self._file.write(json.dumps(trial.evaluated_params))
        self.summarywriter = SummaryWriter(os.path.join(self._filefolder,'tensor_events'))

    def log_trial_result(self, iteration: int, trial: "Trial", result: Dict):
        self._file = open(self._filepath, "at")
        for k, v in list(result.items()):
            if k in self.waive_params:
                del result[k]
                continue
            if isinstance(v, (np.ndarray,)):
                result[k] = v.tolist()
        self._file.write(json.dumps(result))
        self.summarywriter.add_scalar('score', result['score'], len(self.bayesopt._buffered_trial_results)+1)

    def on_trial_complete(self, iteration: int, trials: List["Trial"],
                          trial: "Trial", **info):
        self._file = open(self._filepath, "at")
        self._file.write(str(datetime.datetime.now())+'\n')
        self._file.close()

        self.summarywriter.close()

In [ ]:
smoke_test = False
restore = False
nsample = -1
debug = True
bayesopt = BayesOptSearch(metric="score",
                          mode="min",
                          points_to_evaluate=[{'ntau':0.5, 'nsig': 0.005, "c_ee": 13.25, "c_ei": 12.0,
                                               'c_ie': 9.23, 'c_ii': 2.0, 'tau_e': 10,
                                               'tau_i': 10,'r_e': 1.0,'r_i': 1.0}],
                          random_search_steps=800)
# pdb.set_trace()
if restore:
    bayesopt.restore(os.path.join(restore_folder, 'baye_checkpoint.pkl'))
    # bayesopt._total_random_search_trials=99
if debug:
    sys.path.append('/opt/app-root/src/.local/lib/python3.8/site-packages/ray/tune')
    # pdb.set_trace()
try:
    analysis = tune.run(run_model,
                        verbose=3,
                        name = exp_name,
                        local_dir=ray_folder,
                        search_alg=bayesopt,
                        config=config,
                        metric="score",
                        mode="min",
                        progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
                                                                      parameter_columns=config),
                        callbacks=[CustomLoggerCallback(bayesopt=bayesopt, filefolder=exp_folder, filename= "log_test.txt")],
                        stop={'score':0.08},
                        # resume= 'ERRORED_ONLY', # do this when you whant to resume a stopped experiment option:'ERRORED_ONLY'
                        max_concurrent_trials = 3,
                        num_samples=2 if smoke_test else nsample)
except ValueError as e:
    # TODO: Add the error!
    if str(e) == "blablabla":
        analysis = tune.run(run_model,
                            verbose=3,
                            name = exp_name,
                            local_dir=ray_folder,
                            search_alg=bayesopt,
                            # config=config,
                            metric="score",
                            mode="min",
                            progress_reporter=tune.JupyterNotebookReporter(overwrite=True,metric_columns='score',
                                                                      parameter_columns=config),
                            callbacks=[CustomLoggerCallback(filefolder=exp_folder, filename= "log_test.txt",bayesopt=bayesopt)],
                            stop={'score':0.08},
                            # resume=True, # do this when you whant to resume a stopped experiment
                            max_concurrent_trials = 3,
                            num_samples=2 if smoke_test else nsample)
    elif str(e) ==  "'NoneType' object is not callable":
        pdb.post_mortem()
    else:
        raise
except:
    pdb.post_mortem(sys.exc_info()[2])

Trial name,status,loc,<ray.tune.sample.Float object at 0x7f5ec816b4f0>,<ray.tune.sample.Float object at 0x7f5ec816b580>,<ray.tune.sample.Float object at 0x7f5ec816b610>,<ray.tune.sample.Float object at 0x7f5ec816b6a0>,<ray.tune.sample.Float object at 0x7f5ec816b730>,<ray.tune.sample.Float object at 0x7f5ec816b7c0>,<ray.tune.sample.Float object at 0x7f5ec816b850>,<ray.tune.sample.Float object at 0x7f5ec816b8e0>,<ray.tune.sample.Float object at 0x7f5ec816b9a0>,<ray.tune.sample.Float object at 0x7f5ec816ba30>
run_model_810031dc,RUNNING,10.130.5.143:56726,0.5,0.005,13.25,12,9.23,2,10,10,1,1
run_model_906138d8,RUNNING,10.130.5.143:56910,9.21874,0.0063811,13.4696,9.3254,17.274,9.68448,6.78409,48.7995,0.632739,0.793974
run_model_93c8de4a,RUNNING,10.130.5.143:56973,5.42696,0.00352841,12.8887,7.81405,15.9449,8.07026,11.1826,148.033,0.711386,1.7033
run_model_842bd00a,TERMINATED,10.130.5.143:56759,1.55995,0.00240417,12.8745,9.85214,14.7839,8.79598,90.1673,106.211,0.587125,1.79926
run_model_8756bbaa,TERMINATED,10.130.5.143:56793,1.83405,0.00263642,12.5206,9.90973,15.9893,7.63702,64.7918,43.6844,0.956363,1.28713
run_model_8a9694ca,TERMINATED,10.130.5.143:56832,7.85176,0.00510463,13.1119,7.41848,9.50574,8.09909,88.8622,6.96756,0.799511,1.27135
run_model_8baff6c6,TERMINATED,10.130.5.143:56839,8.08397,0.00969069,13.1075,7.51157,6.78062,9.84666,102.635,66.0229,0.956921,0.646508
run_model_8f5e2a90,TERMINATED,10.130.5.143:56902,6.62522,0.00332902,12.622,8.48553,6.41266,9.72796,82.0065,27.7282,0.967567,1.2801


In [ ]:
# save the current experiment
save_ckp=True
if save_ckp:
    bayesopt.save(os.path.join(exp_folder, 'baye_checkpoint.pkl'))

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)
print("Best result: ", analysis.best_result)
analysis.results_df

## Load the result from file

In [ ]:
from ray.tune import ExperimentAnalysis
analysis = ExperimentAnalysis(exp_folder)

In [ ]:
analysis.get_best_config(metric="score",mode="min",)

In [ ]:
analysis.trials[0].metric_analysis

In [ ]:
min(analysis.results_df.score)